## Creating two sets of random integers between 0 and 10,0000

In [ ]:
__author__ = "Yasaman Emami"
__email__ = ['emami.yasamann@gmail.com','yasaman.emami@sjsu.edu']
__sid__ = "015325557"

In [ ]:
import random

#creating 2 lists of random numbers between start and stop(0,10000) 
start = 0
stop = 100000
first_limit = 250
second_limit = 350

#fisrt random list
first_random_list_int = [random.randint(start, stop) for iter in range(first_limit)]

#second random list and it includes every other fifth element of the random_list_int list so they are not mutually exclusive
second_random_list_int = first_random_list_int[::5] + [random.randint(start, stop) for iter in range(second_limit)]

#print(first_random_list_int)
#print(second_random_list_int)

## Writing generated sets into files

In [ ]:
#create text files for R and S sets
sets = {"S.txt":first_random_list_int, "R.txt":second_random_list_int}

for item in sets.items():
    textfile = open(item[0], "w")
    #print(item[0])
    for element in item[1]:
        #print(element)
        textfile.write(str(element) + "\n")
    textfile.close()   

## Reading text files using pyspark and mapreduce to find set diffrence

In [ ]:
from pyspark import SparkContext

#create Spark context with necessary configuration
sc = SparkContext.getOrCreate()

sets_with_associations=[]

for item in sets.items():
    
    #read data from text file and split each line into one element
    rddfile_1 = sc.textFile(item[0]).flatMap(lambda line: line.split(" "))
    
    #associate each element in each file with the set name and store it in rdd format 
    sets_with_associations.append(rddfile_1.map(lambda x:(int(x),item[0].split(".")[0])).distinct().sortByKey())

#print(sets_with_associations[0].collect())
#print(sets_with_associations[1].collect())

In [ ]:
#concat two rdds value to make one rdd
concat_sets_with_association = sets_with_associations[0] + sets_with_associations[1]
#sortby key values in rdd
sorted_concat_sets_with_association = concat_sets_with_association.sortByKey()
#print(sorted_concat_sets_with_association.collect())


In [ ]:
#for same key, values in final rdd concat set names
res = sorted_concat_sets_with_association.reduceByKey(lambda x,y: x+y ).sortByKey()
#print(res.collect())

In [ ]:
#filters out the values that are only in R and map the values with themselves as requested by assignment
set_difference = res.filter(lambda x: x[1] == 'R' ).map(lambda x: (x[0],x[0])).collect()
print(set_difference)


## Test program

In [ ]:
Start = 0
len_set_difference = len(set_difference)
limit = int(len_set_difference/10)

#creating random index list from the results as big as 10 percent of the result list
random_index_check = [random.randint(Start, len_set_difference) for iter in range(limit)]

#check if the value of randomly picked indexes from set_difference is in R or S set 
for element in random_index_check:
    if set_difference[element][0] in first_random_list_int:
        print("'" + str(element) + "'" +" is in S")
    if set_difference[element][0] in second_random_list_int:
        print("'" + str(element) + "'" + " is in R")